In [1]:
!pip install PyMuPDF
!pip install pypdfium2
!pip install pillow

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pypdfium2 from https://files.pythonhosted.org/packages/25/bd/56d9ec6b9f0fc4e0d95288759f3179f0fcd34b1a1526b75673d2f6d5196f/pypdfium2-4.30.0-py3-none-win_amd64.whl.metadata
     ---------------------------------------- 0.0/48.5 kB ? eta -:--:--
     -------- ------------------------------- 10.2/48.5 kB ? eta -:--:--
     ------------------------ ------------- 30.7/48.5 kB 217.9 kB/s eta 0:00:01
     -------------------------------------- 48.5/48.5 kB 306.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.9 MB 1.5 MB/s eta 0:00:02
   --- ------------------------------------ 0.3/2.9 MB 2.9 MB/s eta 0:00:01
   ------ --------------------------------- 0.5/2.9 MB 3.4 MB/s eta 0:00:01
   ------------ --------------------------- 0.9/2.9 MB 4.8 MB/s eta 0:00:01
   ---------------- --------------


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:

import traceback
from PIL import Image
from io import BytesIO
import pypdfium2 as pdfium
from typing import Optional,List, Dict
import fitz  # PyMuPDF
from pytesseract import image_to_string
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

In [5]:
def convert_pdf_to_images(file_path: str, scale=300/72) -> List[Dict[int, bytes]]:
    pdf_file = pdfium.PdfDocument(file_path)
    page_indices = [i for i in range(len(pdf_file))]

    renderer = pdf_file.render(
    pdfium.PdfBitmap.to_pil,
    page_indices = page_indices,
    scale = scale,
    )

    list_final_images = []

    for i, image in zip(page_indices, renderer):

        image_byte_array = BytesIO()
        image.save(image_byte_array, format='jpeg', optimize=True)
        image_byte_array = image_byte_array.getvalue()
        list_final_images.append(dict({i:image_byte_array}))

    return list_final_images

def getPDFTxt(list_dict_final_images: List[Dict[int, bytes]]) -> str:

  image_list = [list(data.values())[0] for data in list_dict_final_images]
  image_content = []

  for index, image_bytes in enumerate(image_list):
    try:
      image = Image.open(BytesIO(image_bytes))  
      raw_text = str(image_to_string(image))
      
    except Exception:
      raw_text = "OCR failed"
      print(f"OCR failed for image {index}")
      traceback.print_exc()

    image_content.append(raw_text)

  return "\n".join(image_content)

In [15]:
def getPDFTxt_2(file_path):
    # Open the PDF file
    document = fitz.open(file_path)
    text = ""

    # Extract text from each page
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text += page.get_text()

    # Check if extracted text is empty
    if text.strip() == "":
        # Call convert_pdf_to_images function
        images = convert_pdf_to_images(file_path)
        # Call getPDFTxt function with images
        text = getPDFTxt(images)

    return text

In [16]:
result = getPDFTxt_2("sem_annotated.pdf")
result

'7/18/22, 2:12 PM\nEnd Semester Result\nabout:blank\n1/1\nFRANCIS XAVIER ENGINEERING COLLEGE, TIRUNELVELI - 627 003\n(AUTONOMOUS)\nOFFICE OF THE CONTROLLER OF EXAMINATIONS\nResult - April/May 2022\nStudent Name: SYED ABJIMIAH I\nSemester: 04\nRegister Number: 95072056012\nGPA: 9.000\nClass: M.E.- 2020-2022- A\nSEMESTER\nCOURSE CODE\nCREDIT\nGRADE\nGRADE POINT\nRESULT\n04\n19IS4911\n12\nA+\n9\nP\nP - Pass , F - Fail , C - Completed , NC - Not Completed.\nFA\nHOD\nPrincipal\n'

In [17]:
# Now save the result to a .txt file
output_file = "output.txt"

try:
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(result)
    print(f"Text successfully saved to {output_file}")
except Exception as e:
    print(f"Error saving text to file: {e}")

Text successfully saved to output.txt


In [12]:
scanned_result = getPDFTxt_2("scanned_pdf.pdf")
scanned_result

'SYED ABJIMIAH |\n\nSenior Associate - Software Developer\n(AIML / NLP)\n\nPassionate about language and technology, | specialize\nin Natural Language Processing (NLP) and Artificial\nIntelligence Markup Language (AIML). My expertise\nspans developing sophisticated chatbots, optimizing\naudio recording web apps, and seamlessly translating\ncontent into multiple languages. With a keen focus on\nenhancing user experience and delivering precise\ntranslations.\n\nWORK EXPERIENCE\n\nSenior Associate Enterprise sys\n\nSutherland Global PVT LTD\n\n06/2022 - Present Chennai\n\nAchievements/Tasks\n\n© Automatic Question Generation: Leveraged transformer models to generate\nquestions from various sources (PDFs, words, audio, video), achieving a 25%\nincrease in accuracy and a 30% boost in processing speed.\n\n® Optimized a sentiment analysis model , increasing its F1 score from 0.82 to 0.91,\n\nCreated a question-answering chatbot that handled 10,000+ user queries per\nday with a 95% accuracy ra

In [14]:
# Now save the result to a .txt file
output_file = "scanned_pdf_output.txt"

try:
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(scanned_result)
    print(f"Text successfully saved to {output_file}")
except Exception as e:
    print(f"Error saving text to file: {e}")

Text successfully saved to scanned_pdf_output.txt
